## Data

To reduce the training time, we will only use the ticks in Jan, 2020


Preprocess Method:
* Fit the dataframe according to the batch size multiple
* Scaling with Minmax Scaler
* Add technical variables

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import datetime

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib

Selecting previously unselected package libta-lib0.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 7.5MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437801 sha256=055e30608863b83a78cf6bd0fc67176b69219747178b29997c92a972d79e1f69
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [10]:
df_bid = pd.read_csv('/content/drive/MyDrive/ADL final project/data/AAPL.USUSD_Candlestick_1_M_BID.csv')
df_ask = pd.read_csv('/content/drive/MyDrive/ADL final project/data/AAPL.USUSD_Candlestick_1_M_ASK.csv')

df = df_bid.merge(df_ask, on = 'Local time', how = 'inner')
df.head()

,Local time,Open_x,High_x,Low_x,Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Volume_y
0,02.01.2020 09:30:00.000 GMT-0500,296.248,296.268,295.367,295.678,0.7800,296.302,296.303,295.443,295.742,0.6639
1,02.01.2020 09:31:00.000 GMT-0500,295.768,295.818,295.657,295.748,0.0825,295.862,295.882,295.753,295.842,0.0808
2,02.01.2020 09:32:00.000 GMT-0500,295.748,295.778,295.678,295.758,0.0975,295.842,295.842,295.752,295.842,0.0806
3,02.01.2020 09:33:00.000 GMT-0500,295.758,295.838,295.637,295.838,0.4275,295.812,295.882,295.702,295.882,0.4231
4,02.01.2020 09:34:00.000 GMT-0500,295.838,296.897,295.838,296.868,1.8375,295.883,296.953,295.883,296.942,1.5123


In [4]:
df['Open'] = (df['Open_x'] + df['Open_y'])/2
df['Close'] = (df['Close_x'] + df['Close_y'])/2
df['High'] = (df['High_x'] + df['High_y'])/2
df['Low'] = (df['Low_x'] + df['Low_y'])/2

df = df.loc[:, ['Open', 'Close', 'High', 'Low', 'Volume_x', 'Volume_y']]
df

,Open,Close,High,Low,Volume_x,Volume_y
0,296.2750,295.710,296.2855,295.4050,0.7800,0.6639
1,295.8150,295.795,295.8500,295.7050,0.0825,0.0808
2,295.7950,295.800,295.8100,295.7150,0.0975,0.0806
3,295.7850,295.860,295.8600,295.6695,0.4275,0.4231
4,295.8605,296.905,296.9250,295.8605,1.8375,1.5123
...,...,...,...,...,...,...
93178,122.2200,122.185,122.2550,122.1650,3.3619,3.4400
93179,122.1950,122.210,122.2205,122.1350,1.7607,1.6700
93180,122.2200,122.235,122.2455,122.1395,3.7251,3.8800
93181,122.2450,122.230,122.2555,122.1495,4.1460,4.3800


In [5]:
df['MA'] = df['Close'].rolling(window=14).mean()
df['Returns'] = np.log(df['MA']/df['MA'].shift(1))
df = df.dropna()
df.index = np.arange(0, len(df))
df

,Open,Close,High,Low,Volume_x,Volume_y,MA,Returns
0,297.3300,297.325,297.3300,297.3250,0.0225,0.0400,297.035000,0.000388
1,297.2400,297.165,297.2400,297.1600,0.1050,0.0412,297.132857,0.000329
2,297.9000,297.660,297.9250,297.5145,3.0150,2.6786,297.265714,0.000447
3,297.6695,297.785,297.8155,297.5945,3.1800,2.8610,297.403214,0.000462
4,297.7850,297.815,297.9255,297.6850,3.0375,2.3263,297.468214,0.000219
...,...,...,...,...,...,...,...,...
93164,122.2200,122.185,122.2550,122.1650,3.3619,3.4400,122.062857,-0.000053
93165,122.1950,122.210,122.2205,122.1350,1.7607,1.6700,122.056429,-0.000053
93166,122.2200,122.235,122.2455,122.1395,3.7251,3.8800,122.063214,0.000056
93167,122.2450,122.230,122.2555,122.1495,4.1460,4.3800,122.071786,0.000070


In [6]:
from talib import MA_Type

timeperiod = 14

upper, middle, lower = talib.BBANDS(df['Close'], matype=MA_Type.T3)

df['BBand_upper'] = upper
df['BBand_middle'] = middle
df['BBand_lower'] = lower

adx = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod= timeperiod)
dx = talib.DX(df['High'], df['Low'], df['Close'], timeperiod= timeperiod)

macd, macdsignal, macdhist = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
slowk, slowd = talib.STOCH(df['High'], df['Low'], df['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

df['Average Directional Index'] = adx
df['Directional Index'] = dx
df['MACD'] = macd
df['MACD_signal'] = macdsignal
df['MACD hist'] = macdhist
df['stochastic k'] = slowk
df['stochastic d'] = slowd

In [7]:
df['movement'] = 0
df.loc[df['Returns'] > 0, 'movement'] = 1
df.index = np.arange(0, len(df))
df

,Open,Close,High,Low,Volume_x,Volume_y,MA,Returns,BBand_upper,BBand_middle,BBand_lower,Average Directional Index,Directional Index,MACD,MACD_signal,MACD hist,stochastic k,stochastic d,movement
0,297.3300,297.325,297.3300,297.3250,0.0225,0.0400,297.035000,0.000388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,297.2400,297.165,297.2400,297.1600,0.1050,0.0412,297.132857,0.000329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,297.9000,297.660,297.9250,297.5145,3.0150,2.6786,297.265714,0.000447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,297.6695,297.785,297.8155,297.5945,3.1800,2.8610,297.403214,0.000462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,297.7850,297.815,297.9255,297.6850,3.0375,2.3263,297.468214,0.000219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93164,122.2200,122.185,122.2550,122.1650,3.3619,3.4400,122.062857,-0.000053,122.279334,122.036715,121.794096,18.047640,19.461349,-0.019483,-0.026656,0.007173,56.997702,37.459158,0
93165,122.1950,122.210,122.2205,122.1350,1.7607,1.6700,122.056429,-0.000053,122.342990,122.082559,121.822129,17.821065,14.875593,-0.007855,-0.022896,0.015041,83.065749,58.266148,0
93166,122.2200,122.235,122.2455,122.1395,3.7251,3.8800,122.063214,0.000056,122.371346,122.129983,121.888620,17.813164,17.710453,0.003339,-0.017649,0.020988,83.830275,74.631242,1
93167,122.2450,122.230,122.2555,122.1495,4.1460,4.3800,122.071786,0.000070,122.207497,122.171386,122.135275,17.888957,18.874260,0.011672,-0.011785,0.023457,87.270642,84.722222,1


In [8]:
df.to_csv('/content/drive/MyDrive/ADL final project/data/final_input.csv')